In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts
import custom_ts_length

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [3]:
input_num = 0 #this determines the data type via -> ['Wake', 'N1', 'N2','N3', 'REM'][input_num]

In [2]:
%%capture captured_output
#generate all path names and class list(s) etc. 
channels = constants.channel_list
paths , class_list, sleep_night_list , sleep_type_list , participant_id_list = constants.generate_paths_and_info(blue_pebble = True)

In [4]:
%%capture captured_output

t1 = time.time()

folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'
data_type = ['Wake', 'N1', 'N2','N3', 'REM'][input_num]
paths = joblib.load(folder + data_type + '_paths.pkl')

#Load each raw dataframe and convert into a row for the larger dataframe 
overall_dfs_rows = []

for path in paths['selected_paths']:
    data_epo = mne.read_epochs(path)
    df_full = data_epo.to_data_frame()

    #Convert the dataframe into a row
    new_row = {}
    for col in df_full.columns: #iterate through feature columns   
        series = df_full[col] #This gives you the series that you want to append to the new dataframe
        new_row[str(col)] = series
    
    overall_dfs_rows.append(new_row)

df_all_samples = pd.DataFrame(overall_dfs_rows)

t2 = time.time()

time_taken = t2 - t1 #takes around 10 seconds for the Wake Data

In [7]:
#Choose how many minutes of the Raw EEG data you actually want to use
mins = 10
new_length = 256*60*mins #convert minutes to the number of data points
df_snipped = custom_ts_length.customise_df_ts_length(df_all_samples, new_length, impute = False)
df_snipped = df_snipped.drop(columns = ['time', 'condition', 'epoch'] )

In [9]:
# convert into TS_Fresh format Dataframe so that TS Fresh statistics can be calculated
ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(df_snipped, ts_cols = list(df_snipped.columns))

In [17]:
setting = MinimalFCParameters()
setting = EfficientFCParameters()

15-Dec-23 15:41:29 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
15-Dec-23 15:41:29 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!


In [13]:
i = 1

subset_df = ts_fresh_df.iloc[:, :1 + i].copy()
subset_df['time'] = ts_fresh_df['time']

subset_df

,id,Fp1,time
0,0,-1.904406,0
1,0,-3.024036,1
2,0,-10.879476,2
3,0,-9.667392,3
4,0,-3.379244,4
...,...,...,...
3128315,55,-10.067904,66555
3128316,55,-15.889436,66556
3128317,55,-18.451807,66557
3128318,55,-14.736068,66558


n_jobs = 10 seems faster than n_jobs = 55 / 57, I am not sure why that is? <br> 
n_jobs = 10 got to around 37% at 10 minutes, lets see where n_jobs = 3 gets to!

In [21]:
t1 = time.time()
extracted_ts_fresh_df = extract_features(subset_df, column_id = 'id' , column_sort = 'time',  default_fc_parameters=setting, n_jobs = 3)
t2 = time.time()
time_taken = t2-t1
print(time_taken)

Feature Extraction: 100%|██████████| 14/14 [22:04<00:00, 94.58s/it] 

1324.4131066799164


In [16]:
extracted_ts_fresh_df

,Fp1__sum_values,Fp1__median,Fp1__mean,Fp1__length,Fp1__standard_deviation,Fp1__variance,Fp1__root_mean_square,Fp1__maximum,Fp1__absolute_maximum,Fp1__minimum
0,235.235370,-0.381516,0.005105,46080.0,9.430178,88.928258,9.430179,50.931791,50.931791,-38.069487
1,273.533118,-0.033785,0.003816,71680.0,5.744259,32.996516,5.744261,35.544905,35.544905,-23.849265
2,-2150.354824,-2.570979,-0.041999,51200.0,23.248560,540.495549,23.248598,128.414453,128.414453,-88.063352
3,1264.480354,0.040982,0.049394,25600.0,14.971694,224.151630,14.971776,110.966372,110.966372,-82.511484
4,-298.277870,-0.389124,-0.006473,46080.0,9.075626,82.366992,9.075629,96.157790,96.157790,-42.773965
...,...,...,...,...,...,...,...,...,...,...
51,174.529938,-0.218751,0.002622,66560.0,7.133378,50.885084,7.133379,38.756320,38.756320,-27.364053
52,84.657346,-0.248836,0.001181,71680.0,7.101956,50.437773,7.101956,32.031996,33.250250,-33.250250
53,-203.245818,-0.323194,-0.003609,56320.0,8.679205,75.328605,8.679206,49.320537,49.320537,-41.861651
54,-2125.945324,0.020200,-0.027682,76800.0,11.387639,129.678317,11.387672,156.723254,156.723254,-145.610233


In [23]:
( 1324/60 )

22.066666666666666